<a href="https://colab.research.google.com/github/mashdas/nlp_101/blob/master/imdb_review_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten,Dense,GlobalMaxPooling1D,Embedding
from tensorflow.keras import Sequential

In [0]:
import tensorflow_datasets as tfds
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

In [0]:
imdb

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}

In [0]:
train_data,test_data=imdb["train"],imdb["test"]

In [0]:
for x in train_data:
  print(len(x))
  break

2


In [0]:
a,b=None,None
for x,y in train_data:
  print(x)
  print(y)
  a,b=x,y
  break


tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [0]:
print(type(a))
print(b)
##a and b are both tensors..Need to convert them to the proper types
##before tokenizing and training the model

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(0, shape=(), dtype=int64)


In [0]:
training_sentences,training_labels=[],[]
testing_sentences,testing_labels=[],[]
for x,y in train_data:
  training_sentences.append(str(x.numpy()))
  training_labels.append(int(y))

for x,y in test_data:
  testing_sentences.append(str(x.numpy()))
  testing_labels.append(int(y))

training_labels,testing_labels=np.array(training_labels),np.array(testing_labels)

In [0]:
##Building the vocabulary and vectors

tokenizer=Tokenizer(num_words=10000,oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=140,truncating="post")

test_seq=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(test_seq,maxlen=140)

In [0]:
##Building the model
max_len=140
embedding_dim=16
vocab_size=10000
model=Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(Flatten())
model.add(Dense(6,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 140, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 2240)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 13446     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 173,453
Trainable params: 173,453
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(padded, training_labels, e/pochs=10, validation_data=(testing_padded, testing_labels))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5044 - accuracy: 0.7235 - val_loss: 0.3362 - val_accuracy: 0.8522
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2352 - accuracy: 0.9104 - val_loss: 0.3544 - val_accuracy: 0.8482
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0939 - accuracy: 0.9755 - val_loss: 0.4401 - val_accuracy: 0.8359
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0273 - accuracy: 0.9958 - val_loss: 0.4991 - val_accuracy: 0.8374
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0097 - accuracy: 0.9989 - val_loss: 0.5431 - val_accuracy: 0.8403
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0032 - accuracy: 0.9998 - val_loss: 0.5949 - val_accuracy: 0.8392
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.6489 - val_accuracy: 0.8387
Epoch 

In [0]:
##Totally overfitting
##TRY TWEAKING THE VOCAB_SIZE AND PADDING_SIZE...(REDUCE IT PROBABLY)
##Try using Subwords,RNN and LSTM in conjuncion


##Now..to see the word vectors in the embedding space
##Word_index gives us a dictionary with elements as word:some_number
##We need to create a new dict/list that gives us some_number:word
word_index

In [0]:
reverse_word_index=dict([(x,y) for (y,x) in word_index.items()])

In [0]:
reverse_word_index

In [0]:
##As we will be iterating over our review,thats now encoded,
##we need to create afunction that takes in an array and outputs the 
##original human-understandable review
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [0]:
l=model.layers[0]

In [0]:
weights=l.get_weights()[0]

(10000, 16)

In [0]:
##Gives us the weights assigned to each word(10000 total) in our vacabulary
##in the 16 dimensional embedding space.
##You can view this embedding dimensions in http://projector.tensorflow.org/
##Upload the tsv files to check it out


import io

out_v=io.open("vec.tsv",'w',encoding='utf-8')
out_m=io.open("meta.tsv",'w',encoding='utf-8')

for x in range(1,vocab_size):
  word=reverse_word_index[x]
  embeddings=weights[x]
  out_m.write(word+"\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()  

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vec.tsv')
  files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence)
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216]]


In [0]:


encoder = tfds.features.text.SubwordTextEncoder.build_from_corpus(
corpus_generator, target_vocab_size=2**15)
encoder.save_to_file(vocab_filename)

# Load
encoder = tfds.features.text.SubwordTextEncoder.load_from_file(vocab_filename)
ids = encoder.encode("hello world")
text = encoder.decode([1, 2, 3, 4])